# Baseline Verification

Verify the pre-optimized submission.csv scores ~70.65 as expected.

In [ ]:
import pandas as pd
import numpy as np
import math
from numba import njit
import json

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

In [ ]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx = min(mnx, X)
            mxx = max(mxx, X)
            mny = min(mny, Y)
            mxy = max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

In [ ]:
def parse_submission(filepath):
    """Parse submission CSV and return dict of N -> (xs, ys, degs)"""
    df = pd.read_csv(filepath)
    
    # Parse values (remove 's' prefix)
    df['x_val'] = df['x'].str.replace('s', '').astype(float)
    df['y_val'] = df['y'].str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
    
    # Extract N from id (format: NNN_idx)
    df['N'] = df['id'].str.split('_').str[0].astype(int)
    
    configs = {}
    for n, group in df.groupby('N'):
        xs = group['x_val'].values
        ys = group['y_val'].values
        degs = group['deg_val'].values
        configs[n] = (xs, ys, degs)
    
    return configs

In [ ]:
def calculate_total_score(configs, tx, ty):
    """Calculate total score across all N values"""
    total = 0.0
    scores_by_n = {}
    for n in range(1, 201):
        if n in configs:
            xs, ys, degs = configs[n]
            score = score_group(xs, ys, degs, tx, ty)
            scores_by_n[n] = score
            total += score
    return total, scores_by_n

In [ ]:
# Load and score the pre-optimized submission
submission_path = '/home/code/preoptimized/submission.csv'
configs = parse_submission(submission_path)

print(f"Loaded {len(configs)} configurations")
print(f"N values: {min(configs.keys())} to {max(configs.keys())}")

# Verify N=1 has 1 tree, N=200 has 200 trees
for n in [1, 2, 5, 10, 50, 100, 200]:
    xs, ys, degs = configs[n]
    print(f"N={n}: {len(xs)} trees")

In [ ]:
# Calculate total score
total_score, scores_by_n = calculate_total_score(configs, TX, TY)

print(f"\nTotal Score: {total_score:.6f}")
print(f"Expected: ~70.647327")
print(f"\nTop 10 N values by score contribution:")
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:10]:
    print(f"  N={n}: {score:.6f}")

In [ ]:
# Verify score breakdown
print("\nScore breakdown by N range:")
ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores_by_n[n] for n in range(start, end+1))
    print(f"  N={start}-{end}: {range_score:.4f}")

print(f"\nSum check: {sum(scores_by_n.values()):.6f}")

In [ ]:
# Save metrics
import os
os.makedirs('/home/code/experiments/001_baseline', exist_ok=True)

metrics = {
    'cv_score': total_score,
    'scores_by_n': {str(k): v for k, v in scores_by_n.items()}
}

with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Saved metrics to experiments/001_baseline/metrics.json")
print(f"CV Score: {total_score:.6f}")

In [ ]:
# Copy submission to submission folder
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(submission_path, '/home/submission/submission.csv')
print("Copied submission.csv to /home/submission/submission.csv")